# Generating 100K model points

This notebook generates the sample model points 
ued by the "3. Memory-Optimized Mutiprocess Example" notebook.
A DataFrame of 100,000 model points is generated and saved to an Excel file named *model_point_table_100K.xlsx* in the current directory.
The generated model point file can be loaded back by calling pandas' `read_excel` function with `index_col=0`.
The DataFrame can be used with `CashValue_ME` by setting it to `model_point_table` in the `Projection` space.

**Index and Columns:**

* `point_id`(Index): Model point identifier
* `spec_id`: Character identifier ('A','B','C' and 'D') representing the product specs of the model point. `product_spec_table` in `CashValue_ME.Projection` defines the product specs for each identifier.
* `age_at_entry`: Issue age. The samples are distributed uniformly from 20 to 59.
* `sex`: "M" or "F" to indicate policy holder's sex.
* `policy_term`: Policy term in years. The samples are evenly distriubted among 10, 15 and 20.
* `policy_count`: The number of policies. Uniformly distributed from 0 to 100.
* `sum_assured`: Sum assured. The samples are uniformly distributed from 10,000 to 1,000,000.
* `duration_mth`: Months elapsed from the issue til t=0. New business is assumed so 0 for all model points.
* `premium_pp`: Premium per policy.
* `av_pp_init`: Initial account value per policy for existing policies. 0 for all model points.

**Number of model points:**

* 100,000


Click the badge below to run this notebook online on Google Colab. You need a Google account and need to be logged in to it to run this notebook on Google Colab.
[![Run on Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lifelib-dev/lifelib/blob/current/lifelib/libraries/savings/generate_100K_model_points.ipynb)


The next code cell below is relevant only when you run this notebook on Google Colab. It installs lifelib and creates a copy of the library for this notebook.

In [1]:
import sys, os

if 'google.colab' in sys.modules:
    lib = 'savings'; lib_dir = '/content/'+ lib
    if not os.path.exists(lib_dir):
        !pip install lifelib
        import lifelib; lifelib.create(lib, lib_dir)
        
    %cd $lib_dir

In [2]:
import numpy as np
from numpy.random import default_rng  # Requires NumPy 1.17 or newer

rng = default_rng(12345)

# Number of Model Points
MPCount = 10000

# Issue Age (Integer): 20 - 59 year old
age_at_entry = rng.integers(low=20, high=60, size=MPCount)

# Sex (Char)
Sex = ["M", "F"]

sex = np.fromiter(map(lambda i: Sex[i], rng.integers(low=0, high=len(Sex), size=MPCount)), np.dtype('<U1'))

# Policy Term (Integer, modified later for whole life): 10, 15, 20 
policy_term = rng.integers(low=0, high=3, size=MPCount) * 5 + 10

 
# Sum Assured (Float): 10,000 - 1,000,000  
sum_assured = np.round(np.random.uniform(low=100_000, high=1_000_000, size=MPCount), -3)

# Policy Count (Integer): 1
policy_count = np.rint(100 * rng.random(size=MPCount)).astype(int)

In [3]:
import pandas as pd

attrs = [
    "age_at_entry",
    "sex",
    "policy_term",
    "policy_count",
    "sum_assured"
]

data = [
    age_at_entry,
    sex,
    policy_term,
    policy_count,
    sum_assured
]

model_point_table = pd.DataFrame(dict(zip(attrs, data)), index=range(1, MPCount+1))
model_point_table.index.name = "policy_id"

In [4]:
# Spec ID
SpedIDs = "ABCD"

spec_ids = np.fromiter(map(lambda i: SpedIDs[i], rng.integers(low=0, high=len(SpedIDs), size=MPCount)), np.dtype('<U1'))
model_point_table.insert(0, 'spec_id', spec_ids)

# Segment ID
spec_to_seg = {'A': 1, 'B': 2, 'C': 3, 'D': 4}
model_point_table.insert(0, 'liab_id', model_point_table['spec_id'].map(spec_to_seg))


# Modify Policy Term
level_payment = (model_point_table['spec_id'] == 'C')  | (model_point_table['spec_id'] == 'D')
# model_point_table['policy_term'] = model_point_table['policy_term'].mask(whole_life, 9999)

# Set Duration (0 to 20 years in month)

max_duration = np.minimum(model_point_table['policy_term'], 20)
model_point_table['duration_mth'] = np.rint(np.random.uniform(low=0, high=max_duration) * 12).astype(int)

# Premium per Plicy (by adjusting size between single premium(A and B) and level premium(C and D) policies)

level_pp = np.ceil(model_point_table['sum_assured'] / model_point_table['policy_term'] / 12 / 100) * 100

model_point_table.insert(len(model_point_table.columns), 'premium_pp', model_point_table['sum_assured'])
model_point_table['premium_pp'] = model_point_table['premium_pp'].mask(level_payment, level_pp)

# Ininital Account Value per Policy (Normally distributed around accumulated premiums)

acc_prem = model_point_table['premium_pp'].mask(level_payment, model_point_table['premium_pp'] * model_point_table['duration_mth'])
model_point_table['av_pp_init'] = np.rint(np.random.normal(loc=acc_prem, scale=0.2 * acc_prem))

# Change av_pp_init for new biz to 0
model_point_table.loc[model_point_table['duration_mth']==0, 'av_pp_init'] = 0


In [5]:
model_point_table

,liab_id,spec_id,age_at_entry,sex,policy_term,policy_count,sum_assured,duration_mth,premium_pp,av_pp_init
policy_id,,,,,,,,,,
1,3,C,47,M,10,62,284000.0,42,2400.0,105955.0
2,1,A,29,M,20,75,240000.0,221,240000.0,233056.0
3,3,C,51,F,10,80,331000.0,72,2800.0,250868.0
4,4,D,32,F,20,42,733000.0,2,3100.0,6391.0
5,3,C,28,M,15,60,541000.0,92,3100.0,316281.0
...,...,...,...,...,...,...,...,...,...,...
9996,3,C,47,M,20,82,731000.0,166,3100.0,577240.0
9997,3,C,30,M,15,82,819000.0,68,4600.0,333197.0
9998,1,A,45,F,20,78,230000.0,64,230000.0,139156.0


In [6]:
model_point_table.to_excel('alm_model_point_10K.xlsx')